In [15]:
import os 
from Model.AMO import * 

'''
Hướng dẫn:
- Chính với Cường chạy theo từng phần của mình, trước khi chạy thì làm trống file Solution.txt như bấy lâu nay
- Chính chạy 2 task, xong task 1 thì sang task 2 luôn. 2 cái này chạy nhanh 
- Cường chạy xong thì điền kết quả vào cột ACO trong excel 
Note: File có dạng data_gen/... là của bộ data tự sinh. Chính là chạy data tự sinh, gồm 10 bộ thôi. Task 1 là của AMO-ACO, Task 2 là của ACO. 
Chạy xong thì điền kq vào file excel mới. 

Cố gắng chạy xong luôn trong hôm nay đi, này nhanh. Còn nhiều cái phải chạy nữa 
'''

# # --------------------------Chinh--------------------------
# # Task 1 
# file = ['data_gen/gen0.txt', 'data_gen/gen1.txt','data_gen/gen2.txt','data_gen/gen3.txt',
#         'data_gen/gen4.txt','data_gen/gen5.txt','data_gen/gen6.txt','data_gen/gen7.txt',
#         'data_gen/gen8.txt','data_gen/gen9.txt']
# idx = 1
# options = 0
# with open('Data.txt', 'w') as outfile:
#     file_path = file[idx]
#     with open(file_path) as infile:
#         for line in infile:
#             outfile.write(line)
# model = Net3().to(device)
# model.load_state_dict(torch.load('AMO-ACO-100.pt', map_location=torch.device('cpu')))


# # Task 2
# file = ['data_gen/gen0.txt', 'data_gen/gen1.txt','data_gen/gen2.txt','data_gen/gen3.txt',
#         'data_gen/gen4.txt','data_gen/gen5.txt','data_gen/gen6.txt','data_gen/gen7.txt',
#         'data_gen/gen8.txt','data_gen/gen9.txt']
# idx = 0
# options = 2 
# with open('Data.txt', 'w') as outfile:
#     file_path = file[idx]
#     with open(file_path) as infile:
#         for line in infile:
#             outfile.write(line)
# model = Net3().to(device)
# model.load_state_dict(torch.load('AMO-ACO-100.pt', map_location=torch.device('cpu')))





# --------------------------Cuong--------------------------
file = [
     'C206.txt',
 'C207.txt',
 'C104.txt',
 'C105.txt',
 'C107.txt',
  'C208.txt',

 'R112.txt',
 'R107.txt',
 'R110.txt',
 'R207.txt',
 'R210.txt',
 'R209.txt',

 'RC107.txt',
'RC108.txt',
'RC109.txt',
'RC206.txt',
'RC207.txt',
'RC208.txt'
]


 
# Doi reward_coef phu họp 
r_coef = -1 # Cuong 
r_coef = 0 # Cuong 
r_coef = 2 # Chinh 
r_coef = 4 # Chinh 
# --------------------------------------------
idx = 0
options = 0 
with open('Data.txt', 'w') as outfile:
    file_path = 'txt/' + file[idx]
    with open(file_path) as infile:
        for line in infile:
            outfile.write(line)
model = Net3().to(device)
model.load_state_dict(torch.load('AMO-ACO-100.pt', map_location=torch.device('cpu')))


<All keys matched successfully>

In [16]:
import torch
from torch.distributions import Categorical
import random
import itertools
import numpy as np
import copy
import torch
from torch import nn
from torch.nn import functional as F
from copy import deepcopy
import math 
import numpy as np
from Model.Config import *
from Model.AMO import * 
from Ant_k_starts import *
from Normalize_data import *
from Ant import *
from Injection import *
from Cross_Exchange import *
from Local_search import * 
from Draw import *


In [17]:
device = None
cfg = Data_100()
EPS = 1e-10






In [18]:
pyg_data_normalize = normalize_data(cfg)
heuristic_measure, log, topk = model(pyg_data_normalize)
heuristic_measure = heuristic_measure.reshape((cfg.graph_size+1, cfg.graph_size+1))

In [19]:
max_cap, xcoord, ycoord, demand, e_time, l_time, s_time, data = load_data()

data = torch.tensor([[float(x) for x in y] for y in data])
tsp_coordinates = data[:, 1:3] 
demands = torch.tensor(demand, dtype = torch.float32)
time_window = data[:, 4:]
durations = time_window[:, -1] 
distances = gen_distance_matrix(tsp_coordinates, device = device)
if options == 0:
    aco = ACO(distances, demands, time_window, 10, topk, max_cap, options=options, heuristic=heuristic_measure, n_ants=cfg.n_ants)
else:
    aco = ACO(distances, demands, time_window, 10, topk, max_cap, options=options, n_ants=cfg.n_ants)


In [20]:
max_cap, xcoord, ycoord, demand, e_time, l_time, s_time, data = load_data()

CAP=max_cap
colony=Ant(data,CAP,0.7, heuristic_measure)
colony.customer_cord()
colony.euclidean_distance()
colony.width_window()
_ = colony.path_pheromon()

In [21]:
def convert_dict(path): # path: tensor depot 0(m,)
    path += 1
    path = path.to(torch.long)
    zero_indices = torch.where(path == 1)[0]
    zero_indices = zero_indices.tolist()
    path = path.tolist()
    while zero_indices[-1] - zero_indices[-2] == 1:
        zero_indices.pop()
    dict = {}
    for i in range(len(zero_indices) - 1):
        dict[i] = path[zero_indices[i]: zero_indices[i+1] + 1 ]
    return dict

In [22]:
def update_btnt(path, cost, pheromone, effort, k_candidate, prob_to_update, reward_coff, best_cost, d = 2):
    '''
    path: Loi giai tiem nang
    cost: ham cost cua loi giai
    pheromone: aco.pheromone
    effort: so iter truoc khi ket thuc
    ''' 
    if cost < best_cost:
        reward = int(k_candidate * prob_to_update * reward_coff) * effort
    else:
        reward = int(k_candidate * prob_to_update) * effort
    if d == -1:
        reward = 1 
    for route in path.values(): # route
        for l in range(len(route) - 1):
            pheromone[int(route[l] - 1)][int(route[l+1] - 1)] += reward/cost
    reward_coff += d
    return reward_coff, pheromone
    

AMO-ACO

In [23]:
if options == 0:
    import time
    lstt = []
    max_iteration = cfg.epochs 
    max_iteration = 300 
    n_customer = len(data)
    k_candidate = 100
    prob_to_update = 0.05
    best = 1e10
    best_cost = 1e10
    aco.decay = 1
    aco.beta = 0.5
    elitism_set = 0
    reward_coff = 1 
    effort = 0
    final_path = 0
    prob_to_destroy = 0.05
    cnt = 0
    counter = 0
    best_data = []
    best_cost_data = []
    destroy_data = []
    t1 = time.time()


    def save_solution(ants_route, travel_distance, BTNT, max_route, max_travel):
        if len(ants_route.keys()) < max_route:
            BTNT[-1] = ants_route
            max_travel = travel_distance
            max_route = len(ants_route.keys())
        elif len(ants_route.keys()) == max_route and travel_distance < max_travel:
            BTNT[-1] = ants_route
            max_travel = travel_distance
            max_route = len(ants_route.keys())
        return BTNT, max_travel, max_route


    BTNT = [0]
    max_travel = 1e10

    max_route = int(n_customer)
    for k in range(max_iteration):

        paths, costs = aco.run()
        local_path = []
        candidate_values, indexs = torch.topk(costs, k = k_candidate, largest=False)
        candidate_path = paths.T[indexs] # (k * prob_size)
        for i, (value, path) in enumerate(zip(candidate_values, candidate_path)):
            ants_route = convert_dict(path)
            if torch.rand(1) < prob_to_update:
                travel_distance, ants_route = injection(ants_route, colony, 0.5)
                travel_distance, ants_route = cross_exchange(ants_route, colony)
                travel_distance, ants_route = ls_1route(ants_route, colony)
                travel_distance, ants_route = ls3(ants_route)
                BTNT, max_travel, max_route = save_solution(ants_route, travel_distance, BTNT, max_route, max_travel)
                candidate_values[i] = travel_distance
            local_path.append(ants_route)
        
        aco.pheromone *= aco.decay
        value_to_update, index_to_update = torch.topk(candidate_values, k = int(k_candidate * prob_to_update), largest=False)
        for i, j in enumerate(index_to_update):
            path = local_path[j] # dict
            for route in path.values(): # route
                for l in range(len(route) - 1):
                    if i == 0:
                        aco.pheromone[int(route[l] - 1)][int(route[l+1] - 1)] += 1/value_to_update[i]
                    aco.pheromone[int(route[l] - 1)][int(route[l+1] - 1)] += 1/value_to_update[i]
        if torch.min(candidate_values) < best:
            best = torch.min(candidate_values)
            best_path = local_path[index_to_update[0]]
            if k > 0 and elitism_set != 0:
                path, cost = elitism_set
                reward_coff, pheromone = update_btnt(path, cost, aco.pheromone, effort, k_candidate, prob_to_update, reward_coff, best_cost, d = r_coef)
                aco.pheromone = pheromone 
            elitism_set = (best_path, best)  # tuple (path, cost)
            effort = 1
            tries = 0
        
        # Alimentation:
        if tries == 3 and elitism_set != 0:
            tries = 0
            path, cost = elitism_set
            reward_coff, pheromone = update_btnt(path, cost, aco.pheromone, effort, k_candidate, prob_to_update, reward_coff, best_cost, d=r_coef)
            elitism_set = 0
            aco.pheromone = pheromone 

        
        if elitism_set == 0:
            counter += 1

        
        if elitism_set != 0:
            counter = 0
            ants_route, cost = elitism_set
            for _ in range(1):
                travel_distance, ants_route = injection(ants_route, colony, 0.5)
            for _ in range(1):
                travel_distance, ants_route = cross_exchange(ants_route, colony)
            travel_distance, ants_route = local_search(ants_route, colony, n_customer, cfg.q)
            travel_distance, ants_route = ls3(ants_route)
            for _ in range(50):
                travel_distance, ants_route = ls_1route(ants_route, colony)
            BTNT, max_travel, max_route = save_solution(ants_route, travel_distance, BTNT, max_route, max_travel)
            if travel_distance < cost:
                elitism_set = (ants_route, travel_distance)
                effort += 1
                if travel_distance < best_cost:
                    best_cost = travel_distance 
                    final_path = ants_route
            else:
                effort += 1
                tries += 1
        
        if elitism_set == 0 and counter > prob_to_destroy * max_iteration:
            print(destroy)
            travel_distance, ants_route = destroy(final_path, colony, 0.5)
            travel_distance, ants_route = destroy_ls_1route(final_path, colony)
            r = np.random.random()
            if r < 0.5:
                travel_distance, ants_route = ls1(final_path)
            else:
                travel_distance, ants_route = ls2(final_path)
            print("Destroy: ", travel_distance)
            elitism_set = (ants_route, travel_distance)
            counter = 0
            destroy_data.append(k)




        best_data.append(best)
        best_cost_data.append(best_cost)
        print('epoch {}: Best: {}, Alimentation: {}'.format(k, best, best_cost))
        if (k + 1) % 60 == 0:
            lstt.append(best_cost)
    t2 = time.time()
    time_run = t2-t1



epoch 0: Best: 2403.08349609375, Alimentation: 2024.5765600378197
epoch 1: Best: 2195.206298828125, Alimentation: 1904.6838884785568
epoch 2: Best: 2195.206298828125, Alimentation: 1842.9854056129843
epoch 3: Best: 2195.206298828125, Alimentation: 1841.2574730452081
epoch 4: Best: 2195.206298828125, Alimentation: 1840.2619391079463
epoch 5: Best: 2175.20849609375, Alimentation: 1840.2619391079463
epoch 6: Best: 2168.666015625, Alimentation: 1840.2619391079463
epoch 7: Best: 2143.786865234375, Alimentation: 1840.2619391079463
epoch 8: Best: 2143.786865234375, Alimentation: 1840.2619391079463
epoch 9: Best: 2143.786865234375, Alimentation: 1840.2619391079463
epoch 10: Best: 2143.786865234375, Alimentation: 1813.7505088914013
epoch 11: Best: 2143.786865234375, Alimentation: 1794.3733589348321
epoch 12: Best: 2143.786865234375, Alimentation: 1794.373358934832
epoch 13: Best: 2143.786865234375, Alimentation: 1794.373358934832
epoch 14: Best: 2143.786865234375, Alimentation: 1794.37335893483

ACO 

In [24]:
if options == 2:
    import time
    max_iteration = cfg.epochs 
    max_iteration = 200 
    n_customer = len(data)
    k_candidate = 100
    prob_to_update = 0.05
    best = 1e10
    best_cost = 1e10
    aco.decay = 1
    aco.beta = 0.5
    elitism_set = 0
    reward_coff = 1
    effort = 0
    final_path = 0
    prob_to_destroy = 0.05
    cnt = 0
    counter = 0
    best_data = []
    best_cost_data = []
    destroy_data = []
    t1 = time.time()


    def save_solution(ants_route, travel_distance, BTNT, max_route, max_travel):
        if len(ants_route.keys()) < max_route:
            BTNT[-1] = ants_route
            max_travel = travel_distance
            max_route = len(ants_route.keys())
        elif len(ants_route.keys()) == max_route and travel_distance < max_travel:
            BTNT[-1] = ants_route
            max_travel = travel_distance
            max_route = len(ants_route.keys())
        return BTNT, max_travel, max_route


    BTNT = [0]
    max_travel = 1e10

    max_route = int(n_customer)
    for k in range(max_iteration):

        paths, costs = aco.run()
        local_path = []
        candidate_values, indexs = torch.topk(costs, k = k_candidate, largest=False)
        candidate_path = paths.T[indexs] # (k * prob_size)
        for i, (value, path) in enumerate(zip(candidate_values, candidate_path)):
            ants_route = convert_dict(path)
            if torch.rand(1) < prob_to_update:
                travel_distance, ants_route = injection(ants_route, colony, 0.5)
                travel_distance, ants_route = cross_exchange(ants_route, colony)
                travel_distance, ants_route = ls_1route(ants_route, colony)
                BTNT, max_travel, max_route = save_solution(ants_route, travel_distance, BTNT, max_route, max_travel)
                candidate_values[i] = travel_distance
            local_path.append(ants_route)

        aco.pheromone *= aco.decay
        value_to_update, index_to_update = torch.topk(candidate_values, k = int(k_candidate * prob_to_update), largest=False)
        for i, j in enumerate(index_to_update):
            path = local_path[j] # dict
            for route in path.values(): # route
                for l in range(len(route) - 1):
                    if i == 0:
                        aco.pheromone[int(route[l] - 1)][int(route[l+1] - 1)] += 1/value_to_update[i]
                    aco.pheromone[int(route[l] - 1)][int(route[l+1] - 1)] += 1/value_to_update[i]
        if torch.min(candidate_values) < best:
            best = torch.min(candidate_values)
            best_path = local_path[index_to_update[0]]




        best_data.append(best)
        print('epoch {}: Best: {}'.format(k, best))
    t2 = time.time()
    time_run = t2-t1

Optional

In [25]:
# for _ in range(10):
#     ants_route, cost = BTNT[-1], max_travel
#     for _ in range(1):
#         travel_distance, ants_route = injection(ants_route, colony, 0.5)
#     for _ in range(1):
#         travel_distance, ants_route = cross_exchange(ants_route, colony)
#     travel_distance, ants_route = local_search(ants_route, colony, n_customer, cfg.q)
#     travel_distance, ants_route = ls3(ants_route)
#     for _ in range(50):
#         travel_distance, ants_route = ls_1route(ants_route, colony)
#     BTNT, max_travel, max_route = save_solution(ants_route, travel_distance, BTNT, max_route, max_travel)
#     print(max_travel, max_route)

In [26]:
# # final_solution = max(1000*max_route + max_travel, 1000*len(final_path.keys()) + best_cost)
# # final_solution
# if 1000*max_route + max_travel > 1000*len(final_path.keys()) + best_cost:
#     final_route = final_path
#     final_cost = best_cost
#     final_vehicle = len(final_path.keys())
# else:
#     final_route = BTNT[-1]
#     final_cost = max_travel
#     final_vehicle = max_route

In [28]:
with open('Data.txt', 'r') as f:
    name = f.readline()
name.rstrip()

def change_2_save(index, route):
    ans = "Route #{}:".format(index)
    for node in route[1:-1]:
        ans += " " + str(node-1)
    return ans 

# with open('Solution.txt', 'a') as f:
#     f.write(name + '\n')
#     # for id, i in enumerate(final_route.values()):
#     #     f.write(change_2_save(id+1, i) + '\n')
#     # f.write("Time: {}".format(round(time_run,2)) + '\n')
#     f.write("Vehicle: {}".format(final_vehicle)+ '\n')
#     f.write("Distance: {}".format(final_cost)+ '\n')
#     f.write("-----------------------------------------------"+ '\n')

best1, best2, best3, best4, best5 = lstt 
with open('Solution.txt', 'a') as f:
    f.write(name + '\n')
    f.write("Distance: {} - {} - {} - {} - {}".format(best1, best2, best3, best4, best5)+ '\n')
    f.write("-----------------------------------------------"+ '\n')

    

